# Importando as Bibliotecas

In [ ]:
df_bow = pd.DataFrame( titulo_bow , columns = cv.get_feature_names() )

feature_vector_1 = df_bow.loc[0, :]
feature_vector_2 = df_bow.loc[1, :]

calcular_dis_cos(feature_vector_1, feature_vector_2)

In [ ]:
indices_1, valores_1 = calcular_k_maiores(resultado, 1)
indices_10, valores_10 = calcular_k_maiores(resultado, 10)
indices_50, valores_50 = calcular_k_maiores(resultado, 50)

for indices in [indices_1, indices_10, indices_50]:

    # criando uma coluna nova no df
    df_pp[f'k-{len(indices[0])}'] = 0

    for i in range(len(indices)):

        for j in range(len(indices[0])):
            
            if df_pp['ean'].loc[i] == df_pp['ean'].loc[indices[i][j]]:

                df_pp[f'k-{len(indices[0])}'].loc[i] = 1

In [ ]:
def calcular_k_maiores(res, k):

    indices = []
    valores = []

    for i in range(len(res)):
        
        ind = res[i].argsort()[-k:]
        val = res[i][res[i].argsort()[-k:]]

        indices.append( list(reversed(ind)) )
        valores.append( list(reversed(val)) )

    return indices, valores